In [3]:
n = [108, 105, 115, 116]
for i in n:
    print(chr(i))

l
i
s
t


In [5]:
class elefant():
    x = 0
    
    def __init__(self):
        print("I'am constructed..")
    def party(self):
        self.x = self.x + 1
        print("My 'x' is now: ", self.x)
    
    def __del__(self):
        print("I'am destructed. My 'x' was: ", self.x)

bob = elefant()

I'am constructed..
I'am destructed. My 'x' was:  0


In [8]:
bob.party()

My 'x' is now:  3


In [9]:
bob = 5

I'am destructed. My 'x' was:  3


In [2]:
import sqlite3

conn = sqlite3.connect("../../../00_Common_Data_Sets/emaildb.sqlite")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS Counts")

cur.execute("CREATE TABLE Counts(email TEXT, count INTEGER)")

fname = input('Enter file Name: ')
if (len(fname) < 1):
    fname = "../../../00_Common_Data_Sets/mbox-short.txt"
    
fh = open(fname)

for line in fh:
    if not line.startswith("From: "):
        continue
    pieces = line.split()
    email = pieces[1]
    cur.execute("SELECT count FROM Counts WHERE email = ?", (email,))
    row = cur.fetchone()
    if row is None:
        cur.execute("INSERT INTO Counts (email, count) VALUES (?,1)", (email,) )
        
    else:
        cur.execute("UPDATE Counts SET count = count + 1 WHERE email = ?", (email,))
        
    conn.commit()
    
sqlstr = "SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10"

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])
    

cur.close()

Enter file Name: 
cwen@iupui.edu 5
zqian@umich.edu 4
david.horwitz@uct.ac.za 4
louis@media.berkeley.edu 3
gsilver@umich.edu 3
stephen.marquard@uct.ac.za 2
rjlowe@iupui.edu 2
wagnermr@iupui.edu 1
antranig@caret.cam.ac.uk 1
gopal.ramasammycook@gmail.com 1


To get credit for this assignment, perform the instructions below and upload your SQLite3 database here:

(Must have a .sqlite suffix)
Hint: The top organizational count is 536.
You do not need to export or convert the database - simply upload the .sqlite file that your program creates. See the example code for the use of the connect() statement.

Counting Organizations
This application will read the mailbox data (mbox.txt) and count the number of email messages per organization (i.e. domain name of the email address) using a database with the following schema to maintain the counts.

CREATE TABLE Counts (org TEXT, count INTEGER)
When you have run the program on mbox.txt upload the resulting database file above for grading.
If you run the program multiple times in testing or with dfferent files, make sure to empty out the data before each run.

You can use this code as a starting point for your application: http://www.py4e.com/code3/emaildb.py.

The data file for this application is the same as in previous assignments: http://www.py4e.com/code3/mbox.txt.

Because the sample code is using an UPDATE statement and committing the results to the database as each record is read in the loop, it might take as long as a few minutes to process all the data. The commit insists on completely writing all the data to disk every time it is called.

The program can be speeded up greatly by moving the commit operation outside of the loop. In any database program, there is a balance between the number of operations you execute between commits and the importance of not losing the results of operations that have not yet been committed.

In [6]:
import sqlite3

conn = sqlite3.connect("../../../00_Common_Data_Sets/maildb.sqlite")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS Counts")

cur.execute("CREATE TABLE Counts(org TEXT, count INTEGER)")

fname = input('Enter file Name: ')
if (len(fname) < 1):
    fname = "../../../00_Common_Data_Sets/code3/mbox.txt"
    
fh = open(fname)

for line in fh:
    if not line.startswith("From: "):
        continue
    pieces = line.split()
    org = pieces[1].split("@")[1]
    cur.execute("SELECT count FROM Counts WHERE org = ?", (org,))
    row = cur.fetchone()
    if row is None:
        cur.execute("INSERT INTO Counts (org, count) VALUES (?,1)", (org,) )
        
    else:
        cur.execute("UPDATE Counts SET count = count + 1 WHERE org = ?", (org,))
        
    conn.commit()
    
sqlstr = "SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10"

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])
    

cur.close()

Enter file Name: 
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


To get credit for this assignment, perform the instructions below and upload your SQLite3 database here:

(Must have a .sqlite suffix)
You do not need to export or convert the database - simply upload the .sqlite file that your program creates. See the example code for the use of the connect() statement.

Musical Track Database
This application will read an iTunes export file in XML and produce a properly normalized database with this structure:

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
If you run the program multiple times in testing or with different files, make sure to empty out the data before each run.

You can use this code as a starting point for your application: http://www.py4e.com/code3/tracks.zip. The ZIP file contains the Library.xml file to be used for this assignment. You can export your own tracks from iTunes and create a database, but for the database that you turn in for this assignment, only use the Library.xml data that is provided.

To grade this assignment, the program will run a query like this on your uploaded database and look for the data it expects to see:

SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.ID and Track.album_id = Album.id 
        AND Album.artist_id = Artist.id
    ORDER BY Artist.name LIMIT 3
The expected result of the modified query on your database is: (shown here as a simple HTML table with titles)
Select Language​▼
Track	Artist	Album	Genre
Chase the Ace	AC/DC	Who Made Who	Rock
D.T.	AC/DC	Who Made Who	Rock
For Those About To Rock (We Salute You)	AC/DC	Who Made Who	Rock


In [14]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


fname = input('Enter file name: ')
if ( len(fname) < 1 ) :
    fname = "../../../00_Common_Data_Sets/tracks/Library.xml"

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')
    genre = lookup(entry, 'Genre')

    if name is None or artist is None or album is None or genre is None: 
        continue

    print(name, artist, album, genre, count, rating, length)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]
    
    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )

    conn.commit()


Enter file name: 
Dict count: 404
Another One Bites The Dust Queen Greatest Hits Rock 55 100 217103
Asche Zu Asche Rammstein Herzeleid Industrial 79 100 231810
Beauty School Dropout Various Grease Soundtrack 48 100 239960
Black Dog Led Zeppelin IV Rock 109 100 296620
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] Rock 33 100 87118
Circles Bryan Lee Blues Is Funk 54 60 355369
Comfortably Numb Pink Floyd The Wall [Disc 2] Rock 36 100 384130
Crazy Little Thing Called Love Queen Greatest Hits Rock 38 100 163631
Electric Funeral Black Sabbath Paranoid Metal 44 100 293015
Fat Bottomed Girls Queen Greatest Hits Rock 38 100 257515
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock 84 100 353750
Four Sticks Led Zeppelin IV Rock 84 100 284421
Furious Angels Rob Dougan The Matrix Reloaded Soundtrack 54 100 330004
Gelle Bryan Lee Blues Is Blues/R&B 45 60 199836
Going To California Led Zeppelin IV Rock 100 100 215666
Grease Various Grease Soundtrack 42 100 205792
Hand of Doom Bl

Pahrump-Big Water Cusco Natural Wonders Music Sampler 1999 New Age 342 None 191634
Seeker's Quest Cheryl Gunn Natural Wonders Music Sampler 1999 New Age 337 None 236434
Floating To Forever Dean Everson Natural Wonders Music Sampler 1999 New Age 337 None 351686
Open Road Jeff Bailey Relaxing Jazz Jazz 10 None 318563
Ruby Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 13 None 205008
The Angry Mob Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 41 None 288313
Heat Dies Down Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 12 None 237061
Highroyds Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 10 None 199575
Love's Not A Competition (But I'm Winning) Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 11 None 197799
Thank You Very Much Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 11 None 157753
I Can Do It Without You Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 11 None 204199
My Kind Of Guy Kaiser Chiefs Yours Truly, Angry Mob Alt

Katie Hafner: The Origins of the Internet IEEE Computer Society Computing Conversations Podcast 1 None 652460
Larry Smarr: Building Mosaic IEEE Computer Society Computing Conversations Podcast None None 894693
Len Kleinrock: The Theory of Packets IEEE Computer Society Computing Conversations Podcast 1 None 674742
Mitchell Baker: The Mozilla Foundation IEEE Computer Society Computing Conversations Podcast None None 994246
Pooja Sankar: Building the Piazza Collaboration System IEEE Computer Society Computing Conversations Podcast None None 496404
Van Jacobson: Content-Centric Networking IEEE Computer Society Computing Conversations Podcast None None 780251
The Apache Software Foundation IEEE Computer Society Computing Conversations Podcast None None 542484
A Brief History of Packets IEEE Computer Society Computing Conversations Podcast None None 1004643
Discovering JavaScript Object Notation IEEE Computer Society Computing Conversations Podcast None None 650971
Inventing PHP IEEE Compute

Song for Someone U2 Songs of Innocence Rock None None 226763
The Miracle (Of Joey Ramone) U2 Songs of Innocence Rock None None 255382
Iris (Hold Me Close) U2 Songs of Innocence Rock None None 319457
Goodbye Daughters of the Revolution The Black Crowes Warpaint Rock None None 304147
Walk Believer Walk The Black Crowes Warpaint Rock None None 280253
Choice Hotels International Undercover Boss Undercover Boss, Season 2 Reality TV 1 None 2614047
Microchip Jason Farnham YouTube Audio Library Dance & Electronic None None 114336
Anant Agarwal IEEE Computer Society Computing Conversations Podcast None None 494000


To get credit for this assignment, perform the instructions below and enter the code you get here:
 
(Hint: starts with 416)
Instructions
This application will read roster data in JSON format, parse the file, and then produce an SQLite database that contains a User, Course, and Member table and populate the tables from the data file.

You can base your solution on this code: http://www.py4e.com/code3/roster/roster.py - this code is incomplete as you need to modify the program to store the role column in the Member table to complete the assignment.

Each student gets their own file for the assignment. Download this file and save it as roster_data.json. Move the downloaded file into the same folder as your roster.py program.

Once you have made the necessary changes to the program and it has been run successfully reading the above JSON data, run the following SQL command:

SELECT hex(User.name || Course.title || Member.role ) AS X FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY X
Find the first row in the resulting record set and enter the long string that looks like 53656C696E613333.

In [16]:
import json
import sqlite3

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/rosterdb.sqlite')
cur = conn.cursor()

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

fname = input('Enter file name: ')
if len(fname) < 1:
    fname = '../../../00_Common_Data_Sets/roster_data.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0];
    title = entry[1];
    role = entry[2]

    print((name, title, "Role: ", role))

    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''',
        ( user_id, course_id, role ) )

    conn.commit()

Enter file name: 
('Justyna', 'si110', 'Role: ', 1)
('Eidhan', 'si110', 'Role: ', 0)
('Daymian', 'si110', 'Role: ', 0)
('Caidan', 'si110', 'Role: ', 0)
('Connar', 'si110', 'Role: ', 0)
('Hayden', 'si110', 'Role: ', 0)
('Chahat', 'si110', 'Role: ', 0)
('Crystal', 'si110', 'Role: ', 0)
('Zainedin', 'si110', 'Role: ', 0)
('Aref', 'si110', 'Role: ', 0)
('Rhian', 'si110', 'Role: ', 0)
('Shihed', 'si110', 'Role: ', 0)
('Kacy', 'si110', 'Role: ', 0)
('Babyjane', 'si110', 'Role: ', 0)
('Tobi', 'si110', 'Role: ', 0)
('Robyn', 'si110', 'Role: ', 0)
('Oswald', 'si110', 'Role: ', 0)
('Sayeda', 'si110', 'Role: ', 0)
('Luciana', 'si110', 'Role: ', 0)
('Farhaan', 'si110', 'Role: ', 0)
('Kasper', 'si110', 'Role: ', 0)
('Jocelyn', 'si110', 'Role: ', 0)
('Frank', 'si110', 'Role: ', 0)
('Rayne', 'si110', 'Role: ', 0)
('Rivan', 'si110', 'Role: ', 0)
('Layton', 'si110', 'Role: ', 0)
('Jahid', 'si110', 'Role: ', 0)
('Bradly', 'si110', 'Role: ', 0)
('Richie', 'si110', 'Role: ', 0)
('Finnan', 'si106', 'Role: 

('Ruqayah', 'si364', 'Role: ', 0)
('Adana', 'si364', 'Role: ', 0)
('Cherry', 'si364', 'Role: ', 0)
('Jaden', 'si364', 'Role: ', 0)
('Morvern', 'si364', 'Role: ', 0)
('Kitana', 'si364', 'Role: ', 0)
('Bowen', 'si364', 'Role: ', 0)
('Avesta', 'si422', 'Role: ', 1)
('Daood', 'si422', 'Role: ', 0)
('Maram', 'si422', 'Role: ', 0)
('Tanya', 'si422', 'Role: ', 0)
('Sineidin', 'si422', 'Role: ', 0)
('Dalong', 'si422', 'Role: ', 0)
('Lang', 'si422', 'Role: ', 0)
('Kian', 'si422', 'Role: ', 0)
('Jennifer', 'si422', 'Role: ', 0)
('Mario', 'si422', 'Role: ', 0)
('Neshawn', 'si422', 'Role: ', 0)
('Orrick', 'si422', 'Role: ', 0)
('Jaiha', 'si422', 'Role: ', 0)
('Kyron', 'si422', 'Role: ', 0)
('Ameelia', 'si422', 'Role: ', 0)
('Korben', 'si422', 'Role: ', 0)
('Vikki', 'si422', 'Role: ', 0)
('Jad', 'si422', 'Role: ', 0)
('Fintan', 'si422', 'Role: ', 0)
('Miriam', 'si422', 'Role: ', 0)
('Fabian', 'si422', 'Role: ', 0)
('Teagan', 'si422', 'Role: ', 0)
('Sum', 'si422', 'Role: ', 0)
('Hasan', 'si422', 'Ro

Retrieving GEOData
Download the code from http://www.py4e.com/code3/geodata.zip - then unzip the file and edit where.data to add an address nearby where you live - don't reveal where you live. Then run the geoload.py to lookup all of the entries in where.data (including the new one) and produce the geodata.sqlite. Then run geodump.py to read the database and produce where.js. You can run the programs and then scroll back to take your screen shots when the program finishes. Then open where.html to visualize the map. Take screen shots as described below. Make sure that your added location shows in all three of your screen shots.

This is a relatively simple assignment. Don't take off points for little mistakes. If they seem to have done the assignment give them full credit. Feel free to make suggestions if there are small mistakes. Please keep your comments positive and useful. If you do not take grading seriously, the instructors may delete your response and you will lose points.

Please Upload Your Submission:

Screen shot (JPG or PNG - Maximum 1MB) of the geoload.py program output in the command line including your added location (preferably highlighted).
(Please use PNG or JPG files)

Screen shot (JPG or PNG - Maximum 1MB) of the geodump.py program output in the command line including your added location (preferably highlighted).
(Please use PNG or JPG files)

Screen shot (JPG or PNG - Maximum 1MB) of the map zoomed into the location that you added after you open where.html in a browser
(Please use PNG or JPG files)

Enter optional comments below


Make sure each uploaded image file is smaller than 1M. Total upload size limited to 2M

The total number of points for this assignment is 10. You will get up to 3 points from your instructor. You will get up to 4 points from your peers. You will get 1 for each peer assignment you assess. You need to grade a minimum of 3 peer assignments. You can grade up to 5 peer assignments if you like.

In [18]:
#geoload.py
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("../../../00_Common_Data_Sets/code3/geodata/where.data")
count = 0
for line in fh:
    if count > 200 :
        print('Retrieved 200 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    parms = dict()
    parms["address"] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(5)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")


Found in database  AGH University of Science and Technology

Found in database  Academy of Fine Arts Warsaw Poland

Found in database  American University in Cairo

Found in database  Arizona State University

Found in database  Athens Information Technology

Found in database  BITS Pilani

Found in database  Babcock University

Found in database  Banaras Hindu University

Found in database  Bangalore University

Found in database  Baylor University

Found in database  Beijing normal university

Found in database  Belarusian State University

Found in database  Belgrade University

Found in database  Beloit College

Found in database  Belorussian State University

Found in database  Ben Gurion University

Found in database  Bharthidasan University

Found in database  Boston University

Found in database  California Polytechnic State University of San Luis Obispo

Found in database  California State University San Bernardino

Found in database  City of Westminster College

Found in dat


Retrieving http://py4e-data.dr-chuck.net/json?address=Universite+Catholique+de+Louvain&key=42
Retrieved 2407 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+College+Dublin&key=42
Retrieved 1667 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+Munich&key=42
Retrieved 2271 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Akron&key=42
Retrieved 2117 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Alberta&key=42
Retrieved 2021 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Amsterdam&key=42
Retrieved 2050 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Arkansas&key=42
Retrieved 2066 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+At

Retrieved 1678 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Texas&key=42
Retrieved 1795 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Texas+at+Austin&key=42
Retrieved 1795 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Toronto&key=42
Retrieved 2327 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Tuebingen&key=42
Retrieved 2175 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Twente&key=42
Retrieved 1984 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Utah&key=42
Retrieved 2157 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=University+of+Vienna&key=42
Retrieved 2094 characters {    "results" : [  
Pausing for a bit...

Retrieving 

In [19]:
#geodump.py
import sqlite3
import json
import codecs

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('../../../00_Common_Data_Sets/code3/geodata/where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    if not('status' in js and js['status'] == 'OK') : continue

    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    if lat == 0 or lng == 0 : continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'", "")
    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
print("Open where.html to view the data in a browser")

aleja Adama Mickiewicza 30, 30-059 Kraków, Poland 50.06688579999999 19.9136192
Krakowskie Przedmieście 5, 00-068 Warszawa, Poland 52.2394019 21.0150792
AUC Avenue، Road، First New Cairo, Cairo Governorate 11835, Egypt 30.018923 31.499674
Tempe, AZ 85281, USA 33.4242399 -111.9280527
300, 2500 Daniells Bridge Rd, Athens, GA 30606, USA 33.9094132 -83.4603953
Vidya Vihar, Pilani, Rajasthan 333031, India 28.3588163 75.58802039999999
Ilishan-Remo, Nigeria 6.8939569 3.7187158
Ajagara, Varanasi, Uttar Pradesh 221005, India 25.2677203 82.99125819999999
Main Rd, Gnana Bharathi, Bengaluru, Karnataka 560056, India 12.9504351 77.5021998
1301 S University Parks Dr, Waco, TX 76706, USA 31.5497007 -97.1143046
19 Xinjiekou Outer St, Bei Tai Ping Zhuang, Haidian Qu, Beijing Shi, China, 100875 39.9619537 116.3662615
praspiekt Niezaliežnasci 4, Minsk, Belarus 53.8938988 27.5460609
Studentski trg 1, Beograd, Serbia 44.8184518 20.4575913
700 College St, Beloit, WI 53511, USA 42.5030333 -89.0309048
praspiekt

Obala Kulina bana 7, Sarajevo 71000, Bosnia and Herzegovina 43.856883 18.4188848
777-785 W Jefferson Blvd, Los Angeles, CA 90007, USA 34.0236816 -118.2838458
Stellenbosch Central, Stellenbosch, South Africa -33.9328078 18.864447
Ülikooli 18, 50090 Tartu, Estonia 58.3810843 26.7198659
Tehran Province, Tehran, Enghelab Square، Iran 35.7022192 51.39573069999999
Austin, TX 78712, USA 30.2849185 -97.7340567
Austin, TX 78712, USA 30.2849185 -97.7340567
27 Kings College Cir, Toronto, ON M5S, Canada 43.6628917 -79.39565640000001
Geschwister-Scholl-Platz, 72074 Tübingen, Germany 48.5294782 9.043773999999999
5, Drienerlolaan, 7522 NB Enschede, Netherlands 52.23979740000001 6.8499883
201 Presidents Cir, Salt Lake City, UT 84112, USA 40.7649368 -111.8421021
Universitätsring 1, 1010 Wien, Austria 48.21318549999999 16.3600504
Krakowskie Przedmieście 26/28, 00-927 Warszawa, Poland 52.2403463 21.0186012
Seattle, WA 98195, USA 47.65533509999999 -122.3035199
18115 Campus Way NE, Bothell, WA 98011, USA 4

In [21]:
from IPython.display import display
display(filename='../../../00_Common_Data_Sets/code3/geodata/where.html')